In [ ]:
"""
Custom notebook to find the total stellar mass (mass in sink particles) of cluster simulations.
Also plots the average between realisations of the same type, as well as an error bar/
shaded region corresponding the the 1-sigma standard deviation between simulations.

Requires 'totmassinsinks_solenoidals' and 'totmassinsinks_compressives' files to exists for plotting
in working directory.

To generate these files is a bit of a convoluted process:
It requires 'sink*.ev' which contain the whole time evolution of the mass of each sink (i.e. restarts merged).
Since the end of some 'clusterSink*.ev' files is unfinished (due to the code prematurely finishing 
before it finished writing the file), merging them is not straight forward. 
There is a cell at the bottom of the notebook that will remove the last line of each 'clusterSink*.ev' file 
and merge them together. 
Unfortunatley, this is a VERY LONG process. As such it has been intentionally comented out, to prevent 
accidental evaluation of the cell, as it only needs to be done once to get your 'sink*.ev' files.
Directories for each can be specified in the first cell, as a list of strings.

Once you have made the 'sink*.ev' files, you can load them and save the total mass in sink particles
as functions of time, using the second cell. This generates the 'totmassinsinks_solenoidals' and 
'totmassinsinks_compressives' files. If you have already done this, then just use the next cell to simply
load those files, so that you can plot.

Note that the average and standard deviation between simulations is this time done with
np.nanmean and np.std. (Simulations of shorter lengths are padded with nans).
Kinks from faulty restarts or due to some simulations ending earlier than others are less obvious in this plot,
so the average and std of the whole arrays are taken. 
Please note that we again plot only up to a specific point where there is an obvious kink we can't ignore.
Again, this is hard coded (like in 'pdfstats.ipynb'). 

Written by:
David Liptai, Monash University.
2015-2016
"""

In [ ]:
from matplotlib.pyplot import *
import numpy as np
from scipy.stats import ks_2samp as kstest
import itertools
import os
import subprocess
import time
from scipy.interpolate import interp1d
%matplotlib
close('all')

utime       =  1.487E+13/60./60./24./365. 				# in years
t_ff        =  0.806588045/2.*utime						# in years
m_jup       =  0.0009546								# in solar masses

threshold = 1.0e-4
G 		  = 6.67e-8    #cgs
R_sun 	  = 6.963e10   #cm
year	  = 3.15569e7  #s
M_sun	  = 1.9891e33  #g
L_sun	  = 3.846e33   #erg/s
au		  = 1.49598e13 #cm
radius	  = 5*R_sun	   #accretion radius

#Plotting stuff nicely
from matplotlib import gridspec
from IPython.display import display, Math, Latex
import math
from math import sqrt, cos, sin, pi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from matplotlib.ticker import FormatStrFormatter, LinearLocator, NullFormatter, NullLocator, MultipleLocator
import matplotlib.ticker
import matplotlib.colors
from matplotlib.font_manager import FontProperties
from matplotlib import rc, text
plt.close('all')
fig_width_pt = 504   #245.27        # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

fontsize=18
#fig_size = [18,16]
params = {'backend': 'pdf',
          'axes.labelsize': fontsize,
          'lines.markersize': 4,
          'font.size': fontsize,
          'xtick.major.size':8,
          'xtick.minor.size':4,
          'ytick.major.size':8,
          'ytick.minor.size':4,
          'xtick.major.width':2,
          'ytick.major.width':2,
          'xtick.minor.width':1,
          'ytick.minor.width':1,
          'lines.markeredgewidth':1,
          'axes.linewidth':1.2,
          'legend.fontsize': fontsize-3,
          'xtick.labelsize': fontsize-2,
          'ytick.labelsize': fontsize-2,
          'savefig.dpi':200,
          'path.simplify':True,
          'font.family': 'serif',
          'font.serif':'Times',
          'text.latex.preamble': [r'\usepackage{amsmath}'],
          'text.usetex':True,
          'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
                               'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
          #'font.serif':cm,
          'figure.figsize': fig_size}
plt.rcParams.update(params)
plt.clf()
gs = gridspec.GridSpec(1,1)
plt.close('all')

solenoidals = ['sol' +str(i) for i in range(1,8)]
mixed       = ['mix' +str(i) for i in range(1,8)]
compressives= ['comp'+str(i) for i in range(1,8)]

dir_prefix = '/Volumes/dlip1/runs/'
def BASH(command):
    return subprocess.check_output(command,shell=True).decode().strip()

"""
# Old function...no longer used.

# TMIS = Total Mass In Sinks
def massinsinks(dirname):
    dirname = dirname+'/'
    files   = BASH('ls '+dirname+'cluster0*.ev').split('\n')
    # Concatenate cluster0*.ev files (removing the last line)
    lstart = 0
    TMIS   = np.zeros([1e10,2])
    for filename in files:
        try:
            print('Loading:',filename)
            data = np.genfromtxt(filename,skip_footer=1,usecols=(0,17))
            data[:,0] = data[:,0]*utime/t_ff							# time in years, mass in code units (1.9e33 g) (solar mass)
            TMIS[lstart:lstart+len(data),0]=data[:,0]
            TMIS[lstart:lstart+len(data),1]=data[:,1]
            lstart = lstart + len(data)
        except:
            print('failed to complete loading file: '+ filename)
    return TMIS[:lstart,:]
"""

# Function to find the total mass in sink particles for a single simulation.
def massinsinks_V2(dirname):
    #nsinks=BASH('ls '+dirname+'sink*.ev | tail -1')
    #nsinks=int(nsinks.strip('.ev')[-4:])
    filenames = BASH('ls '+dirname+'/sink*.ev')
    filenames = filenames.split('\n')
    i=1
    for fname in filenames:
        #for i in range(1,nsinks+1):
            #fname = dirname+'sink{:0>4}'.format(i)+'.ev'
        try:
            data = np.genfromtxt(fname,usecols=(0,4))
            data[:,0] = data[:,0]*utime/t_ff           # time in years
        except:
            print('failed to load file: '+fname)
        if len(np.shape(data)) > 1:
            mass = data[:,1]                            # mass in code units (1.9e33 g) (solar mass)
            #mass = np.array(mass)
            if i==1:
                t=data[:,0]
                L=len(mass)
                totmass=np.zeros(L)
                massL=np.zeros(L)
                massL[:]=mass[:]
                i = i+1
            else:
                massL=np.zeros(L)
                massL[(L-len(mass)):]=mass
            totmass+=massL
            
    # Combine time column and total mass column
    tmis = np.column_stack((t,totmass))
    return tmis

# Function to load the total mass in sink particles for a collection of simulations into a table.
# (requires massinsinks_V2() function.)
def load_totmassinsinks(folders):
    # Input: list of folder names e.g. [sol1,sol2,....]
    # Output: data array containing: 1st column=time, rest of columns = totmass in sinks for each simulation.
    L_shortest = 1e10
    L_longest  = 0
    data=np.zeros([1e10,len(folders)+1])
    for i in range(0,len(folders)):
        name = folders[i]
        TMIS_i = massinsinks_V2(name)
        L_shortest = min(len(TMIS_i),L_shortest)
        if len(TMIS_i)>L_longest:
            data[:len(TMIS_i),0] = TMIS_i[:,0] #Rewrite time column
        data[:len(TMIS_i),i+1] = TMIS_i[:,1]
        L_longest  = max(len(TMIS_i),L_longest)
        print('Directory:',name,'is complete.')
    shortest_length=L_shortest
    data=data[:L_longest,:]
    return data, shortest_length

In [ ]:
# If you have 'sink*.ev' files, but no 'totmassinsinks_solenoidals' and 'totmassinsinks_compressives'
# files for plotting, use this!
TMIS_sol, lsol = load_totmassinsinks([dir_prefix+name for name in solenoidals])
TMIS_comp,lcomp= load_totmassinsinks([dir_prefix+name for name in compressives])

np.savetxt('totmassinsinks_solenoidals',TMIS_sol)
np.savetxt('totmassinsinks_compressives',TMIS_comp)

In [ ]:
# If you already have the 'totmassinsinks_solenoidals' and 'totmassinsinks_compressives' files
# for plotting, use this!
TMIS_sol=np.loadtxt('totmassinsinks_solenoidals')
TMIS_comp=np.loadtxt('totmassinsinks_compressives')

In [ ]:
plt.close('all')
plt.figure()
plt.plot(TMIS_sol[:,0],TMIS_sol[:,1:])
#plt.plot(TMIS_comp[:,0][:lcomp],TMIS_comp[:,1:][:lcomp],label='a')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Convert zeros to NaNs.
TMIS_sol[TMIS_sol==0]=np.nan
TMIS_comp[TMIS_comp==0]=np.nan
#TMIS_sol[0,0]=0
#TMIS_comp[0,0]=0

In [ ]:
plt.close('all')
#f1 = plt.figure(figsize=fig_size)
plt.figure()
#gs = gridspec.GridSpec(1,1)
#ax0=f1.add_subplot(gs[0])

#t1=TMIS_sol[:lsol,0]
#y1=TMIS_sol[:lsol,1:]
t1=TMIS_sol[:,0]
y1=TMIS_sol[:,1:]
y1_av=np.nanmean(y1,axis=1)
y1_std=np.nanstd(y1,axis=1)

TMIS_SOL=interp1d(t1,y1,axis=0)

t2=TMIS_comp[:,0]
y2=TMIS_comp[:,1:]
y2_av=np.nanmean(y2,axis=1)
y2_std=np.nanstd(y2,axis=1)

#nkink=np.argmax(t2>0.20418) #index beyond which kink occurs
#t2=t2[:nkink]
#y2=y2[:nkink]
#y2_av=y2_av[:nkink]
#y2_std=y2_std[:nkink]

COMP_SIG=interp1d(t2,y2,axis=0)

#Shorten arrays so kinks aren't noticable
end1 = len(t1[t1<1.474])
end2 = len(t2[t2<0.7231])
t1=t1[:end1]
y1=y1[:end1]
y1_av=y1_av[:end1]
y1_std=y1_std[:end1]
t2=t2[:end2]
y2=y2[:end2]
y2_av=y2_av[:end2]
y2_std=y2_std[:end2]

plt.plot(t1,y1_av,'g',linewidth=2,label='Solenoidal')
plt.fill_between(t1, y1_av+y1_std, y1_av-y1_std,color='g',alpha=0.1)
plt.plot(t2,y2_av,'r',linewidth=2,label='Compressive')
plt.fill_between(t2, y2_av+y2_std, y2_av-y2_std,color='r',alpha=0.1)
plt.legend(loc='upper right',frameon=False)
plt.xlabel(r'Time [$t_{\mathrm{ff}}]$')
plt.ylabel(r'Total stellar mass [$\mathrm{M}_{\odot}$]')
#plt.xlim(xmax=max(max(t1),max(t2)))
plt.ylim(ymin=0)
plt.tick_params(which='both',axis='both',pad=8)
plt.tight_layout()
plt.show()

plt.savefig('totmassinsinks.pdf')

#plt.figure()
#plt.plot(TMIS_sol[:,0],TMIS_sol[:,1:])

'''
plt.figure()
t1=sol_rms[:lsol_rms,0]
y1=sol_rms[:lsol_rms,1:]
y1_av=np.mean(y1,axis=1)
y1_std=np.std(y1,axis=1)

SOL_RMS=interp1d(t1,y1,axis=0)

t2=comp_rms[:lcomp_rms,0]
y2=comp_rms[:lcomp_rms,1:]
y2_av=np.mean(y2,axis=1)
y2_std=np.std(y2,axis=1)

np.argmax(t2>0.20418) #index beyond which kink occurs
t2=t2[:nkink]
y2=y2[:nkink]
y2_av=y2_av[:nkink]
y2_std=y2_std[:nkink]
COMP_RMS=interp1d(t2,y2,axis=0)

plt.plot(t1,y1_av,'g',linewidth=2,label='Solenoidal')
plt.fill_between(t1, y1_av+y1_std, y1_av-y1_std,color='g',alpha=0.1)
plt.plot(t2,y2_av,'r',linewidth=2,label='Compressive')
plt.fill_between(t2, y2_av+y2_std, y2_av-y2_std,color='r',alpha=0.1)
plt.legend(loc='upper right')
plt.xlabel(r'$t_{\mathrm{ff}}$')
plt.ylabel(r'$\mathcal{M}$')
plt.xlim(xmax=max(max(t1),max(t2)))
plt.tick_params(which='both',axis='both',pad=8)
plt.show()
'''
print('')

In [ ]:
"""
# IF YOU DO NOT HAVE 'sink*.ev' FILES OR 'totmassinsinks_solenoidals' AND 'totmassinsinks_compressives'
# FILES, use this only ONCE, to generate your 'sink*.ev' files. WARNING: it takes ages.
# Then you can go back to the top and generate 'totmassinsinks_solenoidals' and 'totmassinsinks_compressives'
# for plotting.
#================================================================================
#================================================================================
#================================================================================
def removelastline_sinkfiles(dirname):
    sinkfiles = BASH('ls '+dirname+'clusterSink*.ev')
    sinkfiles = sinkfiles.split('\n')
    for sink in sinkfiles:
        BASH("sed '$d' "+sink+" > "+sink+'.temp')
        
# Remove the last line from each sinkfile and make a temporary file with the same filename appended with .temp
# Note to self: this takes a while....
# Should only need to be done once.
        
for dirname in [dir_prefix+name+'/' for name in solenoidals]:
    removelastline_sinkfiles(dirname)
    print(dirname," done")

for dirname in [dir_prefix+name+'/' for name in compressives]:
    removelastline_sinkfiles(dirname)
    print(dirname," done")
    
    
    
    
#================================================================================
#================================================================================
#================================================================================
def combine_sinkfiles(dirname):
    try:
        simNO=BASH('ls '+dirname+'clusterSink*.ev | tail -1')
        simNO=simNO.strip('.ev')[-3:]
        print('Number of simulations found = '+simNO[-2:])
        nsinks_max=BASH('ls '+dirname+'clusterSink*'+simNO+'.ev | grep -c clusterSink')
    except subprocess.CalledProcessError:
        print('Warning: no sink files found.')
        simNO=''
        nsinks_max='0'
    print(nsinks_max+' sinks found.')
    nsinks_max = int(nsinks_max)

    for i in range(1,nsinks_max+1):
        i=str(i).rjust(4,'0')
        BASH('cat '+dirname+'clusterSink'+i+'N*.ev.temp > '+dirname+'sink'+i+'.ev')

# Combine sinkfiles (with last line removed) of same simulations at restart points
# Note to self: this also takes a while....
# Should only need to be done once.
        
for dirname in [dir_prefix+name+'/' for name in solenoidals]:
    combine_sinkfiles(dirname)
    print(dirname," done")

for dirname in [dir_prefix+name+'/' for name in compressives]:
    combine_sinkfiles(dirname)
    print(dirname," done")

# Should probably also remove the *.ev.temp
"""